In [7]:
from pathlib import Path

from src.retriever import BASE_DIR

In [14]:
BASE_DIR = Path("..").resolve()
print(BASE_DIR)

clapnq_path =  BASE_DIR/"history_selected_rewrite_queries/clapnq_rewritten_last_turn_qwen3_30B.jsonl"
cloud_path = BASE_DIR/"history_selected_rewrite_queries/cloud_rewritten_last_turn_qwen3_30B.jsonl"
fiqa_pth = BASE_DIR/"history_selected_rewrite_queries/fiqa_rewritten_last_turn.jsonl"
govt_path = BASE_DIR/"history_selected_rewrite_queries/govt_rewritten_last_turn.jsonl"
paths = [clapnq_path, cloud_path, fiqa_pth, govt_path]

/Users/sandylin/Challenge in CL


In [16]:
with open(BASE_DIR/"history_selected_rewrite_queries/all_last_turn.jsonl", "w") as fout:
    for path in paths:
        with open(path, "r") as fin:
            for line in fin:
                fout.write(line)